In [55]:
datasets = sorted(["arguana", "dbpedia-entity", "germanquad", "msmarco", "quora", 
            "trec-covid", "climate-fever", "fever", "hotpotqa", "nfcorpus", "scidocs",  
            #"cqadupstack", 
            "fiqa", "nq", "scifact", "webis-touche2020"])

for i in datasets:
    print(i)

arguana
climate-fever
dbpedia-entity
fever
fiqa
germanquad
hotpotqa
msmarco
nfcorpus
nq
quora
scidocs
scifact
trec-covid
webis-touche2020


In [2]:
root = "/gallery_louvre/dayoon.ko/research/sds/src/datasets/"
import pandas as pd
for dataset in datasets:
    df = pd.read_csv(root + dataset + "/qrels/test.tsv", sep="\t")
    if "score" in df.columns:
        score_key = "score"
        corpus_key = "corpus-id"
    elif "rel" in df.columns:
        score_key = "rel"
        corpus_key = "c_id"
    else:
        print(df.columns)
    df = df[df[score_key] >= 1]
    counts = df[corpus_key].value_counts().values.mean()
    print(f"{dataset}: {round(counts, 2)} Q/D")

arguana: 1.0 Q/D
climate-fever: 3.48 Q/D
dbpedia-entity: 1.03 Q/D
fever: 5.29 Q/D
fiqa: 1.0 Q/D
germanquad: 4.42 Q/D
hotpotqa: 1.07 Q/D
msmarco: 1.0 Q/D
nfcorpus: 3.94 Q/D
nq: 1.0 Q/D
quora: 1.0 Q/D
scidocs: 1.23 Q/D
scifact: 1.2 Q/D
trec-covid: 1.41 Q/D
webis-touche2020: 1.01 Q/D


In [9]:
root = "/gallery_louvre/dayoon.ko/research/sds/src/datasets/"
import pandas as pd
for dataset in datasets:
    df = pd.read_csv(root + dataset + "/qrels/test.tsv", sep="\t")
    if "score" in df.columns:
        score_key = "score"
        corpus_key = "corpus-id"
    elif "rel" in df.columns:
        score_key = "rel"
        corpus_key = "c_id"
    df = df[df[score_key] >= 1]
    counts = [i for i in df[corpus_key].value_counts().values if i >=3]
    if len(counts) > 0:
        print(f"{dataset}: {len(counts)} documents with more than 2 related queries")

climate-fever: 275 documents with more than 2 related queries
dbpedia-entity: 60 documents with more than 2 related queries
fever: 715 documents with more than 2 related queries
germanquad: 314 documents with more than 2 related queries
hotpotqa: 84 documents with more than 2 related queries
nfcorpus: 1938 documents with more than 2 related queries
scidocs: 159 documents with more than 2 related queries
scifact: 10 documents with more than 2 related queries
trec-covid: 1422 documents with more than 2 related queries


## Check the # of true corpus 

In [72]:
import os
import json
import pandas as pd
from glob import glob
from tqdm import tqdm

root = f"../sds/src/datasets/"

for root, dirs, files in os.walk(root):
    if "mail" in root or len(files) == 0 or "train" in root or "german" in root:
        continue
    if not files[0].endswith("tsv"):
        continue
    dataset = root.split("/")[-2]
    print(f"\nDataset {dataset}")
    df = pd.DataFrame()
    corpus_pth = f'{"/".join(root.split("/")[:-1])}/corpus.jsonl'
    selected_corpus_pth = corpus_pth.replace(".jsonl", "_selected.jsonl")
    #if os.path.exists(selected_corpus_pth):
    #    continue
    
    for file in files:
        print(f"{root}/{file}")
        df_ = pd.read_csv(f"{root}/{file}", sep="\t")
        #print("current:", len(df))
        #print("new;", len(df_))
        df_ = df_[df_["score"] > 0]
        print("selected;", len(df_))
        df = pd.concat([df, df_])
    corpus_ids = {str(i) for i in set(df["corpus-id"].tolist())}
    print("concat:", len(corpus_ids))
    with open(corpus_pth) as f:
        corpus = [i for i in tqdm(f.readlines()) if str(json.loads(i)["_id"]) in corpus_ids]
        print("selected_corpus:", len(corpus))
    with open(selected_corpus_pth, "w") as f:
        for i in corpus:
            json.dumps(i + "\n")

    


Dataset trec-covid-v2
../sds/src/datasets/trec-covid-v2/qrels/test.tsv
selected; 21538
concat: 14978


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 129179/129179 [00:00<00:00, 137177.53it/s]


selected_corpus: 14978

Dataset nfcorpus
../sds/src/datasets/nfcorpus/qrels/test.tsv
selected; 12334
../sds/src/datasets/nfcorpus/qrels/dev.tsv
selected; 11385
../sds/src/datasets/nfcorpus/qrels/train.tsv
selected; 110575
concat: 3633


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3633/3633 [00:00<00:00, 125628.90it/s]


selected_corpus: 3633

Dataset quora
../sds/src/datasets/quora/qrels/test.tsv
selected; 15675
../sds/src/datasets/quora/qrels/dev.tsv
selected; 7626
concat: 23301


...

selected_corpus: 527341

Dataset climate-fever
../sds/src/datasets/climate-fever/qrels/test.tsv
selected; 4681
concat: 1344


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5416593/5416593 [00:23<00:00, 229356.65it/s]


selected_corpus: 1344

Dataset trec-covid
../sds/src/datasets/trec-covid/qrels/test.tsv
selected; 24673
concat: 17537


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 171332/171332 [00:01<00:00, 155250.22it/s]


selected_corpus: 17537
